In [1]:
import pandas as pd
import sklearn
import numpy as np
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
import torch
from datasets import load_dataset
import tensorflow as tf
import tensorflow_addons as tfa
import gc

from keras import optimizers
import matplotlib.pyplot as plt
from keras import initializers
from keras.utils import np_utils
from keras import regularizers
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import concatenate
from keras.layers import Dense, Flatten, Dropout, Input, BatchNormalization, ReLU
from keras.layers import LSTM, Bidirectional
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score


import keras

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_goss_train = pd.read_pickle('../../Dissertation - Fake News/Embeddings/df_train_roberta_simple.pkl')
df_goss_val = pd.read_pickle('../../Dissertation - Fake News/Embeddings/df_val_roberta_simple.pkl')
df_goss_test = pd.read_pickle('../../Dissertation - Fake News/Embeddings/df_test_roberta_simple.pkl')

In [3]:
df_goss_train.head()

,hidden_state,label,idx
0,"[[-0.033249445, 0.03839188, -0.009663548, -0.1...",0,1934
1,"[[-0.032656625, 0.019969966, 0.003367262, -0.1...",0,1408
2,"[[-0.05037216, 0.030480653, 0.027593914, -0.15...",0,2589
3,"[[-0.053843804, 0.08117483, 0.0040083956, -0.1...",1,5778
4,"[[-0.05073171, 0.040427916, -0.013093643, -0.1...",0,2922


In [4]:
def label_change(df):
    if df['label'] == 0: return 1
    else: return 0

In [5]:
df_goss_train['target'] = df_goss_train.apply(label_change, axis=1)
df_goss_train.head()

,hidden_state,label,idx,target
0,"[[-0.033249445, 0.03839188, -0.009663548, -0.1...",0,1934,1
1,"[[-0.032656625, 0.019969966, 0.003367262, -0.1...",0,1408,1
2,"[[-0.05037216, 0.030480653, 0.027593914, -0.15...",0,2589,1
3,"[[-0.053843804, 0.08117483, 0.0040083956, -0.1...",1,5778,0
4,"[[-0.05073171, 0.040427916, -0.013093643, -0.1...",0,2922,1


In [6]:
df_goss_val['target'] = df_goss_val.apply(label_change, axis=1)
df_goss_test['target'] = df_goss_test.apply(label_change, axis=1)

In [7]:
# Count samples per class: 0-real, 1-fake
classes_zero = df_goss_train[df_goss_train['target'] == 0]
classes_one = df_goss_train[df_goss_train['target'] == 1]

# Convert parts into NumPy arrays for weight computation
zero_numpy = classes_zero['target'].to_numpy()
one_numpy = classes_one['target'].to_numpy()
all_together = np.concatenate((zero_numpy, one_numpy))
unique_classes = np.unique(all_together)

# Compute weights
weights = sklearn.utils.class_weight.compute_class_weight('balanced', unique_classes, all_together)
print(weights)

[0.64400494 2.2360515 ]


C:\Users\labca\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[0 0 0 ... 1 1 1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [8]:
weights_ker = {i : weights[i] for i in range(2)}
weights_ker

{0: 0.6440049443757726, 1: 2.236051502145923}

In [9]:
array_train = df_goss_train[["hidden_state","target"]].to_numpy() 
X_train = np.array(array_train[:,0].tolist())
X_train.shape

(10420, 1, 768)

In [10]:
X_train = np.squeeze(X_train, 1)
X_train.shape

(10420, 768)

In [11]:
y_label = np.array(array_train[:,1].tolist())
y_label.shape

(10420,)

In [12]:
array_val = df_goss_val[["hidden_state","target"]].to_numpy() 
X_val = np.array(array_val[:,0].tolist())
X_val = np.squeeze(X_val, 1)
y_val_label = np.array(array_val[:,1].tolist())

array_test = df_goss_test[["hidden_state","target"]].to_numpy() 
X_test = np.array(array_test[:,0].tolist())
X_test = np.squeeze(X_test, 1)
y_test_label = np.array(array_test[:,1].tolist())

In [13]:
def reset_tensorflow_keras_backend():
    keras.backend.clear_session()
    tf.compat.v1.reset_default_graph()
    _ = gc.collect()

In [14]:
def reinitialize(model):
    for l in model.layers:
        if hasattr(l,"kernel_initializer"):
            l.kernel.assign(l.kernel_initializer(tf.shape(l.kernel)))
        if hasattr(l,"bias_initializer"):
            l.bias.assign(l.bias_initializer(tf.shape(l.bias)))
        if hasattr(l,"recurrent_initializer"):
            l.recurrent_kernel.assign(l.recurrent_initializer(tf.shape(l.recurrent_kernel)))

In [15]:
def run_model(X_train, train_label, X_val, val_label, X_test, test_label, batch_size=32,epochs=100, verbose=1, class_weight=weights_ker, \
              itr=30, shuffle=True):
    
    adam = optimizers.Adam(learning_rate=1e-5)
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    real_precision_list = []
    real_recall_list = []
    real_f1_list = []
    fake_precision_list = []
    fake_recall_list = []
    fake_f1_list = []
    
    macro_precision_list = []
    macro_recall_list = []
    macro_f1_list = []
    accuracy_list = []
    num_epochs = []

    #Define model
    input_text = Input(shape=(768))
    dense_text = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01),\
                kernel_initializer=initializers.he_normal(seed=0))(input_text)
    dense_text = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01),\
                kernel_initializer=initializers.he_normal(seed=0))(dense_text)
    dense_text = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01),\
                kernel_initializer=initializers.he_normal(seed=0))(dense_text)
    output = Dense(1, activation='sigmoid')(dense_text)
    model = Model(inputs=input_text, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics='accuracy')
        
    for i in range(0,itr):
        print("\nTraining for iteration number: ", i)
        reset_tensorflow_keras_backend()
        reinitialize(model)
        history = model.fit(X_train, train_label, validation_data=(X_val, val_label), batch_size=batch_size,epochs=epochs,
                            callbacks=[callback], verbose=verbose, class_weight=class_weight, shuffle=shuffle)
        
        y_pred = model.predict(X_test)
        predictions = list(map(lambda x: 0 if x<0.5 else 1, y_pred))
        
        report = classification_report(y_test_label, predictions, target_names=['real','fake'], output_dict=True)
        real_precision = report['real']['precision'] 
        real_recall = report['real']['recall'] 
        real_f1 = report['real']['f1-score'] 
        fake_precision = report['fake']['precision'] 
        fake_recall = report['fake']['recall'] 
        fake_f1 = report['fake']['f1-score'] 
        
        macro_precision =  report['macro avg']['precision'] 
        macro_recall = report['macro avg']['recall']    
        macro_f1 = report['macro avg']['f1-score']
        accuracy = report['accuracy']

        real_precision_list.append(real_precision)
        real_recall_list.append(real_recall)
        real_f1_list.append(real_f1)
        fake_precision_list.append(fake_precision)
        fake_recall_list.append(fake_recall)
        fake_f1_list.append(fake_f1)
        
        macro_precision_list.append(macro_precision)
        macro_recall_list.append(macro_recall)
        macro_f1_list.append(macro_f1)
        accuracy_list.append(accuracy)
        num_epochs.append(len(history.history['loss']))
        
        
    mean_real_precision = np.mean(real_precision_list)
    mean_real_recall = np.mean(real_recall_list)
    mean_real_f1 = np.mean(real_f1_list)
    mean_fake_precision = np.mean(fake_precision_list)
    mean_fake_recall= np.mean(fake_recall_list)
    mean_fake_f1 = np.mean(fake_f1_list)
    
    mean_macro_precision = np.mean(macro_precision_list)
    mean_macro_recall = np.mean(macro_recall)
    mean_macro_f1 = np.mean(macro_f1_list)
    mean_accuracy = np.mean(accuracy_list)
    mean_epoch = np.mean(num_epochs)
    
    std_real_precision = np.std(real_precision_list)
    std_real_recall = np.std(real_recall_list)
    std_real_f1 = np.std(real_f1_list)
    std_fake_precision = np.std(fake_precision_list)
    std_fake_recall= np.std(fake_recall_list)
    std_fake_f1 = np.std(fake_f1_list)
    
    std_macro_precision = np.std(macro_precision_list)
    std_macro_recall = np.std(macro_recall)
    std_macro_f1 = np.std(macro_f1_list)
    std_accuracy = np.std(accuracy_list)

    print (report)
    
    print("The standard deviation on real precision : ", std_real_precision)
    print("The standard deviation on real recall: ", std_real_recall)
    print("The standard deviation on real f1: ", std_real_f1)
    print("The standard deviation on fake precision : ", std_fake_precision)
    print("The standard deviation on fake recall: ", std_fake_recall)
    print("The standard deviation on macro precision: ", std_macro_precision)
    print("The standard deviation on macro recall: ", std_macro_recall)
    print("The standard deviation on macro_f1: ", std_macro_f1)
    print("The standard deviation on macro accuracy: ", std_accuracy)
    print("Average number of epochs", mean_epoch)
    
    return real_precision_list, real_recall_list, real_f1_list, fake_precision_list, fake_recall_list, fake_f1_list,\
    macro_precision_list, macro_recall_list, macro_f1_list, accuracy_list

In [16]:
real_precision, real_recall, real_f_score, fake_precision, fake_recall,\
fake_f1, macro_precision, macro_recall, macro_f1, accuracy = run_model(X_train, \
y_label, X_val, y_val_label, X_test, y_test_label, verbose=0, itr=30)


Training for iteration number:  0
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  1
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  2
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  3
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  4
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  5
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  6
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  7
41/41 [==============================] - 3s 1ms/step

Training for iteration number:  8
41/41 [==============================] - 3s 1ms/step

Training for iteration number:  9
41/41 [==============================] - 0s 1ms/step

Training for iteration number:  10
41/41 [==============================] - 0s 1ms/step

Training for iteration number:

In [17]:
print("Real Precision:\n ", real_precision)
print("Real Recall:\n ", real_recall)
print("Real F1-Score:\n ",  real_f_score) 

Real Precision:
  [0.8940594059405941, 0.9021852237252862, 0.8988648090815273, 0.9001029866117405, 0.8983572895277208, 0.8995943204868154, 0.8962075848303394, 0.8989690721649485, 0.9010416666666666, 0.8943170488534397, 0.8931068931068931, 0.8977732793522267, 0.8918387413962635, 0.8982706002034588, 0.8960645812310797, 0.8929633300297324, 0.898876404494382, 0.9013360739979445, 0.8998968008255934, 0.8974874371859296, 0.8952095808383234, 0.8951048951048951, 0.8968968968968969, 0.9001029866117405, 0.9045064377682404, 0.8913894324853229, 0.8945273631840795, 0.9030883919062833, 0.8899707887049659, 0.8899707887049659]
Real Recall:
  [0.8861629048086359, 0.8508341511285574, 0.8547595682041217, 0.8577036310107949, 0.858684985279686, 0.8704612365063789, 0.8812561334641805, 0.8557409224730128, 0.8488714425907753, 0.8802747791952895, 0.8773307163886163, 0.8704612365063789, 0.8900883218842002, 0.8665358194308145, 0.8714425907752699, 0.8842001962708538, 0.8635917566241413, 0.8606476938174681, 0.85574

In [18]:
print("Fake Precision:\n ", fake_precision)
print("Fake Recall:\n ", fake_recall)
print("Fake F1-Score:\n ", fake_f1)

Fake Precision:
  [0.6040955631399317, 0.5555555555555556, 0.5568862275449101, 0.5632530120481928, 0.5623100303951368, 0.583596214511041, 0.5980066445182725, 0.5585585585585585, 0.5510204081632653, 0.5933333333333334, 0.5860927152317881, 0.580952380952381, 0.6083916083916084, 0.575, 0.5801282051282052, 0.5986394557823129, 0.5709876543209876, 0.5696969696969697, 0.5598802395209581, 0.5909090909090909, 0.5946843853820598, 0.5927152317880795, 0.5953947368421053, 0.5632530120481928, 0.5256064690026954, 0.6156583629893239, 0.5973154362416108, 0.5302197802197802, 0.6195652173913043, 0.6195652173913043]
Fake Recall:
  [0.6232394366197183, 0.6690140845070423, 0.6549295774647887, 0.6584507042253521, 0.6514084507042254, 0.6514084507042254, 0.6338028169014085, 0.6549295774647887, 0.6654929577464789, 0.6267605633802817, 0.6232394366197183, 0.6443661971830986, 0.6126760563380281, 0.647887323943662, 0.6373239436619719, 0.6197183098591549, 0.6514084507042254, 0.6619718309859155, 0.6584507042253521, 0

In [19]:
print("Macro Precision:\n ", macro_precision)
print("Macro Recall:\n ", macro_recall)
print("Macro F1-Score:\n ", macro_f1)
print("Accuracy Score:\n ", accuracy)

Macro Precision:
  [0.7490774845402629, 0.728870389640421, 0.7278755183132187, 0.7316779993299667, 0.7303336599614287, 0.7415952674989282, 0.7471071146743059, 0.7287638153617535, 0.7260310374149659, 0.7438251910933865, 0.7395998041693406, 0.7393628301523039, 0.750115174893936, 0.7366353001017294, 0.7380963931796425, 0.7458013929060227, 0.7349320294076849, 0.7355165218474571, 0.7298885201732758, 0.7441982640475102, 0.7449469831101916, 0.7439100634464872, 0.7461458168695011, 0.7316779993299667, 0.7150564533854679, 0.7535238977373233, 0.7459213997128451, 0.7166540860630317, 0.7547680030481352, 0.7547680030481352]
Macro Recall:
  [0.7547011707141771, 0.7599241178177998, 0.7548445728344553, 0.7580771676180735, 0.7550467179919557, 0.7609348436053021, 0.7575294751827946, 0.7553352499689008, 0.757182200168627, 0.7535176712877856, 0.7502850765041673, 0.7574137168447388, 0.7513821891111141, 0.7572115716872383, 0.7543832672186208, 0.7519592530650043, 0.7575001036641833, 0.7613097624016918, 0.7570

In [1]:
import numpy as np
macro_f1_mean = np.mean([0.7518059198809633, 0.741393164875593, 0.7390996464222226, 0.7427674084709261, 0.7408310817060364, 0.7502143144634255, 0.7520267955695962, 0.7398699329293286, 0.7385248458092265, 0.7484146985894882, 0.7446220389957083, 0.747462345896208, 0.7507444938475856, 0.745694702648345, 0.7454823199439047, 0.7487783487916573, 0.7447167562299141, 0.7464499692581401, 0.7412207875082208, 0.7508286588475268, 0.7498225893079926, 0.7490251072888858, 0.7518270935728106, 0.7427674084709261, 0.7297960333514619, 0.7525445187809203, 0.7500118848729338, 0.7309345010655872, 0.7520824605502026, 0.7520824605502026])
macro_f1_mean

0.7457280762831979

In [2]:
acc_mean = np.mean([0.8288564850345357, 0.8112049117421335, 0.8112049117421335, 0.8142747505755947, 0.8135072908672295, 0.8227168073676132, 0.827321565617805, 0.8119723714504988, 0.8089025326170376, 0.8250191864927091, 0.8219493476592479, 0.8211818879508825, 0.829623944742901, 0.8188795088257866, 0.8204144282425173, 0.8265541059094398, 0.8173445894090561, 0.8173445894090561, 0.8127398311588642, 0.8250191864927091, 0.8257866462010744, 0.8250191864927091, 0.8265541059094398, 0.8142747505755947, 0.7966231772831927, 0.8319263238679969, 0.8265541059094398, 0.7989255564082886, 0.8326937835763623, 0.8326937835763623])
acc_mean 

0.8199027884369406

In [3]:
real_precision_mean = np.mean([0.8940594059405941, 0.9021852237252862, 0.8988648090815273, 0.9001029866117405, 0.8983572895277208, 0.8995943204868154, 0.8962075848303394, 0.8989690721649485, 0.9010416666666666, 0.8943170488534397, 0.8931068931068931, 0.8977732793522267, 0.8918387413962635, 0.8982706002034588, 0.8960645812310797, 0.8929633300297324, 0.898876404494382, 0.9013360739979445, 0.8998968008255934, 0.8974874371859296, 0.8952095808383234, 0.8951048951048951, 0.8968968968968969, 0.9001029866117405, 0.9045064377682404, 0.8913894324853229, 0.8945273631840795, 0.9030883919062833, 0.8899707887049659, 0.8899707887049659])
real_precision_mean

0.8970693703972764

In [6]:
real_recall_mean = np.mean([0.8861629048086359, 0.8508341511285574, 0.8547595682041217, 0.8577036310107949, 0.858684985279686, 0.8704612365063789, 0.8812561334641805, 0.8557409224730128, 0.8488714425907753, 0.8802747791952895, 0.8773307163886163, 0.8704612365063789, 0.8900883218842002, 0.8665358194308145, 0.8714425907752699, 0.8842001962708538, 0.8635917566241413, 0.8606476938174681, 0.8557409224730128, 0.8763493621197253, 0.8802747791952895, 0.8792934249263984, 0.8792934249263984, 0.8577036310107949, 0.8272816486751717, 0.8940137389597644, 0.8822374877330716, 0.8321884200196271, 0.8969578017664377, 0.8969578017664377])
real_recall_mean

0.8695780176643767

In [5]:
real_f1_mean = np.mean([0.8900936421882699, 0.8757575757575758, 0.8762575452716298, 0.8783919597989951, 0.8780732563973908, 0.884788029925187, 0.8886689757545769, 0.8768225238813474, 0.8741788782213239, 0.887240356083086, 0.885148514851485, 0.8839063278525162, 0.8909626719056976, 0.8821178821178821, 0.8835820895522388, 0.8885601577909271, 0.8808808808808808, 0.8805220883534137, 0.8772635814889337, 0.8867924528301887, 0.8876793666501733, 0.887128712871287, 0.88800792864222, 0.8783919597989951, 0.8641722193746797, 0.8926996570308672, 0.8883399209486166, 0.8661899897854954, 0.8934506353861192, 0.8934506353861192])
real_f1_mean

0.8829840138926042

In [7]:
fake_precision_mean = np.mean([0.6040955631399317, 0.5555555555555556, 0.5568862275449101, 0.5632530120481928, 0.5623100303951368, 0.583596214511041, 0.5980066445182725, 0.5585585585585585, 0.5510204081632653, 0.5933333333333334, 0.5860927152317881, 0.580952380952381, 0.6083916083916084, 0.575, 0.5801282051282052, 0.5986394557823129, 0.5709876543209876, 0.5696969696969697, 0.5598802395209581, 0.5909090909090909, 0.5946843853820598, 0.5927152317880795, 0.5953947368421053, 0.5632530120481928, 0.5256064690026954, 0.6156583629893239, 0.5973154362416108, 0.5302197802197802, 0.6195652173913043, 0.6195652173913043])
fake_precision_mean

0.5800423905666319

In [8]:
fake_recall_mean = np.mean([0.6232394366197183, 0.6690140845070423, 0.6549295774647887, 0.6584507042253521, 0.6514084507042254, 0.6514084507042254, 0.6338028169014085, 0.6549295774647887, 0.6654929577464789, 0.6267605633802817, 0.6232394366197183, 0.6443661971830986, 0.6126760563380281, 0.647887323943662, 0.6373239436619719, 0.6197183098591549, 0.6514084507042254, 0.6619718309859155, 0.6584507042253521, 0.6408450704225352, 0.6302816901408451, 0.6302816901408451, 0.6373239436619719, 0.6584507042253521, 0.6866197183098591, 0.6091549295774648, 0.6267605633802817, 0.6795774647887324, 0.602112676056338, 0.602112676056338])
fake_recall_mean

0.6416666666666664

In [9]:
fake_f1_mean = np.mean([0.6135181975736568, 0.6070287539936102, 0.6019417475728155, 0.6071428571428572, 0.6035889070146819, 0.6156405990016639, 0.6153846153846154, 0.6029173419773096, 0.6028708133971291, 0.6095890410958904, 0.6040955631399317, 0.6110183639398998, 0.6105263157894737, 0.609271523178808, 0.6073825503355705, 0.6089965397923875, 0.6085526315789473, 0.6123778501628664, 0.605177993527508, 0.6148648648648648, 0.6119658119658119, 0.6109215017064847, 0.6156462585034014, 0.6071428571428572, 0.5954198473282442, 0.6123893805309734, 0.6116838487972509, 0.595679012345679, 0.6107142857142858, 0.6107142857142858])
fake_f1_mean

0.6084721386737922